In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

%load_ext autoreload
%autoreload 2

In [ ]:
from PIL import Image
import numpy as np
import torch
from torchvision.transforms.functional import center_crop
from tokenizers import Tokenizer
import matplotlib.pyplot as plt

# Video related: 
import cv2
from IPython.display import Video

from fourm.data.multimodal_dataset_folder import MultiModalDatasetFolder
from fourm.models.fm import FM
from fourm.vq.vqvae import VQVAE, DiVAE
from fourm.models.generate import GenerationSampler, build_chained_generation_schedules, init_empty_target_modality, init_full_input_modality, custom_text
# from utils.generation_abstract_functions import create_generation_schedule_rgb_to_others
from fourm.data.modality_transforms import RGBTransform, DepthTransform, MetadataTransform
from fourm.data.modality_info import MODALITY_INFO, MODALITY_TRANSFORMS
from fourm.utils.plotting_utils import decode_dict, visualize_bboxes, plot_text_in_square
from fourm.utils import denormalize, IMAGENET_INCEPTION_MEAN, IMAGENET_INCEPTION_STD, IMAGENET_DEFAULT_STD, IMAGENET_DEFAULT_MEAN
from fourm.data.modality_transforms import SemsegTransform
from fourm.data.image_augmenter import CenterCropImageAugmenter
from torchvision import transforms
from fourm.data.modality_transforms import UnifiedDataTransform
from fourm.data.dataset_utils import SubsampleDatasetWrapper
from fourm.data.masking import UnifiedMasking
from einops import rearrange
from utils.semseg_helper_utils import semseg_to_rgb, plot_rgb2semseg, get_dataset, get_semseg_metrics, total_intersect_and_union, intersect_and_union, mean_iou, mean_dice, eval_metrics, tokens_per_target_dict, autoregression_schemes_dict, cfg_schedules_dict
from tqdm import tqdm
import matplotlib.colors as mcolors

# The flag below controls whether to allow TF32 on matmul. This flag defaults to False in PyTorch 1.12 and later.
torch.backends.cuda.matmul.allow_tf32 = True
# The flag below controls whether to allow TF32 on cuDNN. This flag defaults to True.
torch.backends.cudnn.allow_tf32 = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.set_grad_enabled(False)

In [ ]:
text_tok = Tokenizer.from_file('toks/text_tokenizer_4m_wordpiece_30k.json')

toks = {
    'tok_rgb': DiVAE.from_pretrained('EPFL-VILAB/4M_tokenizers_rgb_16k_224-448').eval().to(device),
}

In [ ]:
# initalizing the RGB transform class
rgb_transform = RGBTransform(imagenet_default_mean_and_std=False)
img_pil = rgb_transform.load('data/processed/train/rgb/dish_1558031526/dish_1558031526.png')
img_pil = rgb_transform.preprocess(img_pil)
img_pil = center_crop(img_pil, (min(img_pil.size), min(img_pil.size))).resize((224,224))
img = rgb_transform.postprocess(img_pil).unsqueeze(0).to(device)

In [ ]:
import numpy as np
import torch

# 1. Définissez le chemin de votre fichier .npy
npy_file_path = 'data/processed/train/rgb_tok/dish_1558031526/dish_1558031526.npy' # Ceci est une chaîne

# 2. Chargez les données du fichier .npy en utilisant np.load()
# Cela chargera les tokens sous forme d'un tableau NumPy.
# En supposant que save_vq_tokens.py a sauvegardé les tokens avec une forme comme (n_crops, sequence_length)
try:
    all_crop_tokens_numpy = np.load(npy_file_path)
    print(f"Fichier NumPy chargé avec succès. Forme : {all_crop_tokens_numpy.shape}")
except FileNotFoundError:
    print(f"ERREUR : Fichier non trouvé à {npy_file_path}")
    # Gérez l'erreur ici, par exemple en sortant du script ou en levant une exception
    raise
except Exception as e:
    print(f"Une erreur est survenue lors du chargement du fichier NumPy : {e}")
    raise

# 3. Sélectionnez le crop désiré (si n_crops > 1)
# Par exemple, pour le premier crop (center crop) :
CROP_INDEX_TO_VISUALIZE = 0
if CROP_INDEX_TO_VISUALIZE >= all_crop_tokens_numpy.shape[0]:
    print(f"ERREUR : L'index de crop {CROP_INDEX_TO_VISUALIZE} est hors limites pour {all_crop_tokens_numpy.shape[0]} crops.")
    raise IndexError("Index de crop invalide")

selected_tokens_numpy = all_crop_tokens_numpy[CROP_INDEX_TO_VISUALIZE] # Forme : (sequence_length,)
print(f"Tokens pour le crop {CROP_INDEX_TO_VISUALIZE} sélectionnés. Forme : {selected_tokens_numpy.shape}")


# 4. Convertissez le tableau NumPy en tenseur PyTorch
# Les tokens VQVAE sont généralement des entiers (IDs).
tokenized_rgb_tensor = torch.from_numpy(selected_tokens_numpy).long() # Convertit en torch.LongTensor

# 5. Remodelez le tenseur pour qu'il ait la forme attendue par decode_tokens.
# La méthode decode_tokens attend généralement (Batch, H_token_map, W_token_map).
# Vous devez connaître H_tok et W_tok pour votre modèle.
# Par exemple, si votre tokenizer produit une carte de tokens de 14x14 :
H_tok = 14 # À remplacer par la hauteur réelle de votre carte de tokens
W_tok = 14 # À remplacer par la largeur réelle de votre carte de tokens

if tokenized_rgb_tensor.shape[0] != H_tok * W_tok:
    print(f"ERREUR : La longueur de la séquence de tokens ({tokenized_rgb_tensor.shape[0]}) "
          f"ne correspond pas à H_tok*W_tok ({H_tok*W_tok}).")
    print("Veuillez vérifier les valeurs de H_tok et W_tok, ou la validité du fichier de tokens.")
    raise ValueError("Incompatibilité de forme des tokens.")

# Ajoutez une dimension de batch (B=1) et remodelez en (1, H_tok, W_tok)
tokenized_rgb_prepared = tokenized_rgb_tensor.reshape(1, H_tok, W_tok)
print(f"Tokens préparés pour le décodage. Forme : {tokenized_rgb_prepared.shape}")

# Assurez-vous que le tenseur est sur le bon device (celui de votre modèle toks['tok_rgb'])
# Supposons que votre modèle est sur 'cuda' s'il est disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenized_rgb_prepared = tokenized_rgb_prepared.to(device)

In [ ]:
tokenized_rgb_prepared

In [ ]:
reconstructed_rgb = toks['tok_rgb'].decode_tokens(tokenized_rgb_prepared, image_size=224, timesteps=19)

In [ ]:
# reconstructed_rgb
# Create a figure with two subplots (1 row, 2 columns)
fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# Display the original image on the left
axes[0].imshow(denormalize(img, mean=IMAGENET_INCEPTION_MEAN, std=IMAGENET_INCEPTION_STD)[0].permute(1, 2, 0).cpu())
axes[0].set_title("Original Image")
axes[0].axis("off")  # Hide axes

# Display the reconstructed image on the right
axes[1].imshow(denormalize(reconstructed_rgb, mean=IMAGENET_INCEPTION_STD, std=IMAGENET_INCEPTION_STD)[0].permute(1, 2, 0).cpu())
axes[1].set_title("Reconstructed Image")
axes[1].axis("off")  # Hide axes

# Show the figure
plt.tight_layout()
plt.show()